# Gensim을 이용한 LDA 실습 코드
* 출처: [딥 러닝을 이용한 자연어 처리 입문 / 19-02 잠재 디리클레 할당 (Latent Dirichlet Allocation)](https://wikidocs.net/30708)
* 본 코드는 아래와 같은 순서로 구성되어 있습니다.
    1. 뉴스그룹 데이터 불러오기 및 전처리 (LSA 코드와 동일)
    2. Gensim 이용하여 LDA 모델링

## 01. 데이터 로드 & 전처리

In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

'''1. load data'''
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

'''2. preprocessing'''
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

'''3. remove stopwords'''
# 불용어 데이터셋이 없을 경우 아래 코드의 주석을 해제하고 실행하기
# nltk.download('stopwords')
stop_words = stopwords.words('english') # NLTK로부터 불용어 불러오기
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words]) # 불용어 제거

In [2]:
# 훈련용 데이터 확인
tokenized_doc[:5]

0    [well, sure, story, seem, biased., disagree, s...
1    [yeah,, expect, people, read, faq,, etc., actu...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal,,...
4    [well,, change, scoring, playoff, pool., unfor...
Name: clean_doc, dtype: object

In [3]:
# 각각의 단어들을 (단어의 정수 인코딩 값, 해당 단어의 빈도수) 형태로 저장
from gensim import corpora

dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 2), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 2), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 2), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1)]


In [4]:
# 66이라는 정수 인코딩 된 숫자가 인코딩 전에는 어떤 단어였는지 확인하기
print(dictionary[66])


bye-bye,


In [5]:
# 총 학습된 단어의 개수 확인
len(dictionary)


181856

## 02. LDA 모델 훈련 (Gensim 이용)

In [6]:
import gensim
NUM_TOPICS = 20 # 20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
# 한 토픽당 출력할 단어 개수 지정
topics = ldamodel.print_topics(num_words=4)

# 맨 앞에 있는 숫자는 토픽 번호로, 현재 토픽을 20개로 제한하였기 때문에 0부터 19까지 할당
# 각 단어 앞에 붙은 수치는 해당 토픽에 대한 기여도
for topic in topics:
    print(topic)

(0, '0.014*"period" + 0.011*"----" + 0.010*"power" + 0.009*"---------------"')
(1, '0.015*"team" + 0.015*"game" + 0.010*"games" + 0.010*"play"')
(2, '0.016*"anyone" + 0.014*"would" + 0.014*"please" + 0.013*"know"')
(3, '0.003*"hong" + 0.002*"kong" + 0.002*"championships" + 0.002*"answer:"')
(4, '0.007*"would" + 0.006*"people" + 0.005*"many" + 0.004*"government"')
(5, '0.012*"would" + 0.010*"people" + 0.009*"think" + 0.007*"like"')
(6, '0.013*"entries" + 0.004*"hawks" + 0.004*"cubs" + 0.003*"caps"')
(7, '0.005*"jumper" + 0.004*"slave" + 0.003*"jumpers" + 0.002*"esdi"')
(8, '0.024*"armenian" + 0.018*"armenians" + 0.018*"turkish" + 0.008*"turkey"')
(9, '0.008*"chicago" + 0.006*"boston" + 0.006*"detroit" + 0.006*"minnesota"')
(10, '0.007*"radar" + 0.004*"candida" + 0.004*"stealth" + 0.003*"gateway"')
(11, '0.005*"*******" + 0.004*"alcohol" + 0.003*"present." + 0.003*"(205)"')
(12, '0.003*"twin" + 0.003*"patterns" + 0.002*"--------------------------------------------------------------------

## 03. 문서별 토픽 분포 보기

In [8]:
# 상위 5개 문서에 대해 토픽 분포 확인
# (숫자, 확률)은 토픽 번호와 해당 토픽이 문서에서 차지하는 분포도를 의미함
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(4, 0.34551287), (5, 0.3215774), (17, 0.31999916)]
1 번째 문서의 topic 비율은 [(2, 0.14627048), (4, 0.25416204), (5, 0.57923603)]
2 번째 문서의 topic 비율은 [(4, 0.16899686), (5, 0.5783655), (15, 0.063890666), (17, 0.17515764)]
3 번째 문서의 topic 비율은 [(2, 0.3231347), (4, 0.1818687), (5, 0.17374781), (13, 0.1679064), (15, 0.015675435), (18, 0.12761773)]
4 번째 문서의 topic 비율은 [(1, 0.25283885), (2, 0.05036209), (5, 0.30843422), (6, 0.2531329), (13, 0.07076233), (14, 0.03847841)]


In [24]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = pd.concat([topic_table, pd.DataFrame([int(topic_num), round(prop_topic,4), topic_list]).T], ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [25]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,4,0.3455,"[(4, 0.3455353), (5, 0.32155597), (17, 0.31999..."
1,1,5,0.5792,"[(2, 0.14627257), (4, 0.2541641), (5, 0.5792319)]"
2,2,5,0.5784,"[(4, 0.16900979), (5, 0.57835346), (15, 0.0638..."
3,3,2,0.3231,"[(2, 0.3231304), (4, 0.18179756), (5, 0.173639..."
4,4,5,0.3084,"[(1, 0.25284672), (2, 0.0503644), (5, 0.308386..."
5,5,5,0.5935,"[(5, 0.5935031), (10, 0.36552846)]"
6,6,5,0.3305,"[(2, 0.17577219), (5, 0.3304737), (10, 0.15528..."
7,7,5,0.3728,"[(4, 0.08824316), (5, 0.37281448), (13, 0.2845..."
8,8,5,0.5445,"[(5, 0.5444543), (13, 0.14496103), (16, 0.2862..."
9,9,13,0.5004,"[(2, 0.11892111), (4, 0.11527285), (5, 0.15554..."
